In [2]:
from custom_fuzzy_ops import ImpliesReichenbachSigmoidal
import torch
import torch.nn as nn
import numpy as np
import matplotlib.pyplot as plt
import ltn
from globals_ltn import Godel, KleeneDienes, Goguen, Reichenbach, Luk

import pytorch_lightning as pl

In [9]:
class MLP(pl.LightningModule):
    def __init__(self, input_size, hidden_sizes, output_size):
        super().__init__()
        self.input_size = input_size
        self.hidden_sizes = hidden_sizes
        self.output_size = output_size
        self.sigmoid = nn.Sigmoid()

        self.layers = nn.ModuleList()
        self.layers.append(nn.Linear(input_size, hidden_sizes[0]))
        for hidden_size_in, hidden_size_out in zip(hidden_sizes, hidden_sizes[1:]):
            self.layers.append(nn.Linear(hidden_size_in, hidden_size_out))
        self.layers.append(nn.Linear(hidden_sizes[-1], output_size))

    def forward(self, x):
        for layer in self.layers[:-1]:
            x = torch.relu(layer(x))
        x = self.layers[-1](x)
        x = self.sigmoid(x)
        return x

In [35]:
class LTNDataModule(pl.LightningDataModule):
    def __init__(self, constants, positive_examples, negative_examples, axioms):
        super().__init__()
        self.constants = constants
        self.positive_examples = positive_examples
        self.negative_examples = negative_examples
        self.axioms = axioms

    def setup(self, stage = None):
        # Convert data to PyTorch tensors
        self.constants_tensor = torch.tensor(self.constants)
        self.positive_examples_tensor = torch.tensor(self.positive_examples)
        self.negative_examples_tensor = torch.tensor(self.negative_examples)

    def train_dataloader(self):
        # Create a dataloader for training
        return torch.utils.data.DataLoader(
            self.constants_tensor,
            batch_size=1,
            shuffle=True
        )

    def val_dataloader(self):
        # Create a dataloader for validation (optional)
        return torch.utils.data.DataLoader(
            self.constants_tensor,
            batch_size=1,
            shuffle=False
        )

    def test_dataloader(self):
        # Create a dataloader for testing (optional)
        return torch.utils.data.DataLoader(
            self.constants_tensor,
            batch_size=1,
            shuffle=False
        )

    def prepare_axioms(self):
        # Convert axioms to LTN formulas
        ltn_axioms = []
        for axiom in self.axioms:
            # Parse the axiom string and create an LTN formula
            ltn_axiom = ltn.parse(axiom)
            ltn_axioms.append(ltn_axiom)
        return ltn_axioms

In [ ]:
constants

In [37]:
LTNDataModule()

TypeError: __init__() missing 4 required positional arguments: 'constants', 'positive_examples', 'negative_examples', and 'axioms'

In [15]:
input_size = 8
hidden_sizes = [8]
output_size = 1
model = MLP( input_size, hidden_sizes, output_size)
P = ltn.Predicate(model)

In [16]:
P

Predicate(model=MLP(
  (sigmoid): Sigmoid()
  (layers): ModuleList(
    (0): Linear(in_features=8, out_features=8, bias=True)
    (1): Linear(in_features=8, out_features=1, bias=True)
  )
))

In [32]:
a = ltn.Constant(torch.rand(8))
x = ltn.Variable('x1', torch.rand(2,8))
x

Variable(value=tensor([[0.6836, 0.9543, 0.1705, 0.1109, 0.0632, 0.0883, 0.5481, 0.4561],
        [0.6227, 0.0242, 0.1598, 0.6791, 0.2878, 0.1410, 0.1140, 0.6312]]), free_vars=['x1'])

In [33]:
P(x)

LTNObject(value=tensor([0.5668, 0.5748], grad_fn=<ViewBackward0>), free_vars=['x1'])